### This program corresponds to the third experiment in the article, which is used to solve the Helmholtz equation on the bounded domain with a smooth boundary

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import scipy.special as scp
from torch.autograd import Variable as v
import time
import math

import sys
sys.path.append('../Models')
sys.path.append('../Utils')
import importlib



import Network as Nw
import BoundaryCondition as Bdc
import Loss_functions as lf
import DomainSetting as Ds
import GenerateIntegralMatrix as IM
import Training as Tn


<module 'Training' from './Models\\Training.py'>

In [168]:
# Set problem domain and boundary condition
# Generate integral points on boundary and the integral matrix of the fundamental solution 

# k                         The wavenumber
# sample_num                The number of the sample points in boundary
# sample_x                  The coordinate of the sample points in boundary
# sample_u_r, sample_u_i    The boundary condition
# rprime                    The tangent vectors of sample points with parametric equation in boundary
# normal                     The exterior normal of the sample points in boundary

example = 'ex3-2'
if example == 'ex3-2':
    domain_type = 'smoothdomain'
    problem_type ='Helmholtz4'
    layer = 1
    sample_num = 800
    sample_x, normal, rprime = Ds.petaldomain(sample_num)
    sample_u_r, sample_u_i = Bdc.exact_sol(sample_x[:,0],sample_x[:,1],example)

G1_r , G1_i= IM.G_bd(domain_type,problem_type,layer,sample_x,rprime,normal)

net1 = Nw.MLP(40,2,2)
loss_func = lf.Helmholtz_loss()
optimizer = torch.optim.Adam(net1.parameters(net1),lr=0.0001)


In [169]:
# Training
Epoch, each_epoch = 5000, 1000
net1,loss_all = Tn.Helmholtz_training(net1,sample_x,sample_u_r,sample_u_i,G1_r,G1_i,loss_func,optimizer,Epoch,each_epoch)

loss, epoch, computation time: 1.007797 0 0.0200
loss, epoch, computation time: 0.015692 1000 19.9245
loss, epoch, computation time: 0.001264 2000 22.5700
loss, epoch, computation time: 0.000174 3000 18.4274
loss, epoch, computation time: 0.000067 4000 15.5372
loss, epoch, computation time: 0.000042 5000 20.7255


In [170]:
# Randomly generate points x_in in the problem domain and the corresponding exact solution
sample = 1000
theta_in = torch.rand(sample,1)*2*np.pi
r_in = torch.rand(sample,1).sqrt()*(9/20-1/9*torch.cos(5*theta_in))*0.99
x_in = torch.cat((torch.cos(theta_in), torch.sin(theta_in)),1)*r_in 

u_in_r, u_in_i = Bdc.exact_sol(x_in[:,0],x_in[:,1],example)

In [171]:
# Generate integral matrix of points x_in and calculate the relative L2 errir 
G1_in_r,G1_in_i = IM.G_in(problem_type,layer,x_in,sample_x,rprime,normal)
u_green_r , u_green_i = -(G1_in_r@sample_h_r-G1_in_i@sample_h_i) , -(G1_in_r@sample_h_i+G1_in_i@sample_h_r)
print('Relaive L2 error:','%.4f'%(((((u_green_r-u_in_r)**2).sum()+((u_green_r-u_in_r)**2).sum())/((u_in_r**2).sum()+(u_in_r**2).sum())).sqrt()))

Relaive L2 error: 0.0075
